In [2]:
# importing SparkSession
from pyspark.sql import SparkSession

# loading row and datetime
from pyspark.sql.types import Row
from datetime import datetime

# creating spark session
spark = SparkSession.builder\
                     .appName('Analyzing Airlines Data')\
                     .getOrCreate()

In [3]:
# reading and loading airlines dataset
airlines = spark.read\
                .format('csv')\
                .option('header', 'true')\
                .load('../datasets/airlines.csv')
airlines.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [4]:
# creating tempview of the data
airlines.createOrReplaceTempView('airlines')

In [5]:
# printing column of dataset
airlines = spark.sql('SELECT * FROM airlines')
airlines.columns

['Code', 'Description']

In [6]:
# showing the data
airlines.show(5)

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
+-----+--------------------+
only showing top 5 rows



In [7]:
# reading and loading flights dataset
flights = spark.read\
                .format('csv')\
                .option('header', 'true')\
                .load('../datasets/flights.csv')
flights.printSchema()

root
 |-- date: string (nullable = true)
 |-- airlines: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- departure: string (nullable = true)
 |-- departure_delay: string (nullable = true)
 |-- arrival: string (nullable = true)
 |-- arrival_delay: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)



In [8]:
# creating tempview of the data
flights.createOrReplaceTempView('flights')

In [9]:
# printing column of dataset
flights = spark.sql('SELECT * FROM flights')
flights.columns

['date',
 'airlines',
 'flight_number',
 'origin',
 'destination',
 'departure',
 'departure_delay',
 'arrival',
 'arrival_delay',
 'air_time',
 'distance']

In [10]:
# showing the data
flights.show(2)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            1|   JFK|        LAX|     0854|          -6.00|   1217|         2.00|  355.00| 2475.00|
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
only showing top 2 rows



In [11]:
# counting data from datasets
flights.count(), airlines.count()

(476881, 1579)

In [12]:
# getting count using sql queries
flights_count = spark.sql('SELECT COUNT(*) FROM flights')
airlines_count = spark.sql('SELECT COUNT(*) FROM airlines')
# these are also data frames
flights_count, airlines_count

(DataFrame[count(1): bigint], DataFrame[count(1): bigint])

In [13]:
# tabular form of dataframe
flights_count.show(), airlines_count.show()

+--------+
|count(1)|
+--------+
|  476881|
+--------+

+--------+
|count(1)|
+--------+
|    1579|
+--------+



(None, None)

In [14]:
# getting actual count
flights_count.collect()[0][0], airlines_count.collect()[0][0]
# same count as previous

(476881, 1579)

In [15]:
# calculating total distance covered by all flights
total_distance_df = spark.sql('SELECT distance FROM flights')\
                         .agg({'distance':'sum'})\
                         .withColumnRenamed('sum(distance)', 'total_distance')
# total distance
total_distance_df, total_distance_df.show()

+--------------+
|total_distance|
+--------------+
|  3.79052917E8|
+--------------+



(DataFrame[total_distance: double], None)

In [16]:
# getting all delays flight details of 2014
all_delays_2014 = spark.sql(
                     'SELECT date, airlines, flight_number, departure_delay ' +
                     'FROM flights WHERE departure_delay > 0 AND year(date) = 2014')
# displaying delay flights
all_delays_2014.show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-01|   19805|            2|          14.00|
|2014-04-01|   19805|            4|          25.00|
|2014-04-01|   19805|            6|         126.00|
|2014-04-01|   19805|            7|         125.00|
|2014-04-01|   19805|            8|           4.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



In [17]:
# displaying deatils
all_delays_2014.createOrReplaceTempView('all_delays')
all_delays_2014.orderBy(all_delays_2014.departure_delay.desc()).show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-27|   20366|         5246|          99.00|
|2014-04-27|   19393|         2948|          99.00|
|2014-04-27|   20366|         5365|          99.00|
|2014-04-26|   19977|          616|          99.00|
|2014-04-27|   20366|         6030|          99.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



In [18]:
# getting delay count
delay_count = spark.sql('SELECT COUNT(departure_delay) FROM all_delays')
delay_count.show()

+----------------------+
|count(departure_delay)|
+----------------------+
|                179015|
+----------------------+



In [19]:
# getting count
delay_count.collect()[0][0]

179015

In [21]:
# calculating delay percentage
delay_percent = delay_count.collect()[0][0] / flights_count.collect()[0][0] * 100
print('Percentage of flights delay')
delay_percent

Percentage of flights delay


37.53871510922012

In [23]:
# getting delay per airlines
delay_per_airlines = spark.sql('SELECT airlines, departure_delay FROM flights')\
                          .groupBy('airlines')\
                          .agg({'departure_delay':'avg'})\
                          .withColumnRenamed('avg(departure_delay)', 'departure_delay')
delay_per_airlines.orderBy(delay_per_airlines.departure_delay.desc()).show(5)

+--------+------------------+
|airlines|   departure_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows



In [31]:
# creating view
delay_per_airlines.createOrReplaceTempView('delay_per_airlines')
delay_per_airlines = spark.sql('SELECT * FROM delay_per_airlines ORDER BY departure_delay DESC')
delay_per_airlines.show(5)

+--------+------------------+
|airlines|   departure_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows



In [35]:
# joining to get airlines company name
delay_per_airlines = spark.sql('SELECT * FROM delay_per_airlines ' +
                               'JOIN airlines ON airlines.code = delay_per_airlines.airlines ' +
                               'ORDER BY departure_delay DESC'
                              )
delay_per_airlines.show(5)

+--------+------------------+-----+--------------------+
|airlines|   departure_delay| Code|         Description|
+--------+------------------+-----+--------------------+
|   19393|13.429567657134724|19393|Southwest Airline...|
|   20366|12.296210112379818|20366|ExpressJet Airlin...|
|   19977| 8.818392620527979|19977|United Air Lines ...|
|   20436| 8.716275167785234|20436|Frontier Airlines...|
|   20409|  8.31110357194785|20409| JetBlue Airways: B6|
+--------+------------------+-----+--------------------+
only showing top 5 rows

